In [1]:
! pip install -U accelerate
! pip install -U transformers
! pip install tensorboardx



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 29 04:46:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4500               On  | 00000000:03:00.0 Off |                  Off |
| 30%   36C    P8              17W / 200W |      1MiB / 20470MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of our model. Finally, we'll
use `gradio` to build a flashy demo of our fine-tuned model.

In [4]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-pla5fi6t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-pla5fi6t
  Resolved https://github.com/huggingface/transformers to commit ba56ed0869eb4bbeb1c04af7f62a04350150e8d4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[no

## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple.
We can download and prepare the Common Voice splits in just one line of code.

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

Since Hindi is very low-resource, we'll combine the `train` and `validation`
splits to give approximately 8 hours of training data. We'll use the 4 hours
of `test` data as our held-out test set:

In [40]:
from datasets import load_dataset, DatasetDict

mun_dataset = load_dataset("audiofolder", data_dir=r"data")
mun_dataset = mun_dataset["train"].train_test_split(test_size=0.1)

Resolving data files:   0%|          | 0/21191 [00:00<?, ?it/s]

Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [41]:
mun_dataset = mun_dataset.remove_columns(["Unnamed: 0"])

print(mun_dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 19070
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 2119
    })
})


We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [42]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium")

In [43]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="hi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
print(mun_dataset["train"][0])

{'audio': {'path': '/workspace/data/female_14809.wav', 'array': array([3.05175781e-05, 3.05175781e-05, 3.05175781e-05, ...,
       3.05175781e-05, 0.00000000e+00, 3.05175781e-05]), 'sampling_rate': 44100}, 'text': 'सारतेगे चि वायुसेनाको ता: मिआदो बीटीए बानोगा।'}


In [46]:
from datasets import Audio

mun_dataset = mun_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [47]:
print(mun_dataset["train"][0])

{'audio': {'path': '/workspace/data/female_14809.wav', 'array': array([2.08903184e-05, 3.14010176e-05, 3.18827551e-05, ...,
       2.82596411e-05, 3.26804475e-05, 2.20771526e-05]), 'sampling_rate': 16000}, 'text': 'सारतेगे चि वायुसेनाको ता: मिआदो बीटीए बानोगा।'}


In [48]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute input length
    batch["input_length"] = len(batch["audio"])

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["text"]).input_ids

    # compute labels length
    batch["labels_length"] = len(tokenizer(batch["text"], add_special_tokens=False).input_ids)
    batch["labels_length"] = len(batch["labels"])
    return batch

MAX_DURATION_IN_SECONDS = 30.0
max_input_length = MAX_DURATION_IN_SECONDS * 16000

def filter_inputs(input_length):
    """Filter inputs with zero input length or longer than 30s"""
    return 0 < input_length < max_input_length

max_label_length = 448

def filter_labels(labels_length):
    """Filter label sequences longer than max length (448)"""
    return labels_length < max_label_length

In [49]:
mun_dataset = mun_dataset.map(prepare_dataset, remove_columns=mun_dataset.column_names["train"], num_proc=1)
mun_dataset= mun_dataset.filter(filter_inputs, input_columns=["input_length"])

mun_dataset = mun_dataset.filter(filter_labels, input_columns=["labels_length"])

Map:   0%|          | 0/19070 [00:00<?, ? examples/s]

Map:   0%|          | 0/2119 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19070 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2119 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19070 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2119 [00:00<?, ? examples/s]

In [51]:
mun_dataset = mun_dataset.remove_columns(["input_length","labels_length"])

In [52]:
mun_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 19069
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2119
    })
})

## Training and Evaluation

In [53]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [54]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [55]:
import evaluate

metric = evaluate.load("wer")

In [56]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [57]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
model.generation_config.language = "hi"

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [58]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the Training Configuration

In [59]:
import gc
torch.cuda.empty_cache()
gc.collect()

317

In [60]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-mundari",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [61]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=mun_dataset["train"],
    eval_dataset=mun_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [62]:
processor.save_pretrained(training_args.output_dir)

[]

In [63]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.314000,0.324280,66.307600
1000,0.262200,0.266693,58.514529
1500,0.192700,0.238663,52.569985
2000,0.172200,0.221896,51.532682
2500,0.116700,0.212376,48.247889
3000,0.108000,0.203464,45.229071
3500,0.092000,0.196999,43.812754
4000,0.047600,0.205504,43.493583


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/do

TrainOutput(global_step=4000, training_loss=0.22023291343450546, metrics={'train_runtime': 30370.0581, 'train_samples_per_second': 2.107, 'train_steps_per_second': 0.132, 'total_flos': 6.530952863711232e+19, 'train_loss': 0.22023291343450546, 'epoch': 3.36})

In [ ]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!zip -r /content/whisper-medium-santali.zip /content/whisper-medium-santali/

	zip warning: missing end signature--probably not a zip file (did you
	zip warning: remember to use binary mode when you transferred it?)
	zip warning: (if you are trying to read a damaged archive try -F)

zip error: Zip file structure invalid (/content/whisper-medium-santali.zip)


In [ ]:
from google.colab import files
print('foo')

foo


In [ ]:
files.download("/content/whisper-medium-santali.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp whisper-medium-santali.zip '/content/drive/My Drive/TTS'
!ls -lt '/content/drive/My Drive/'

total 5249171
drwx------ 2 root root       4096 Nov 14 17:51  TTS
-rw------- 1 root root 5372116992 Nov 14 17:40  whisper-medium-santali.zip
drwx------ 2 root root       4096 Aug  5  2022  yamiDrive
drwx------ 2 root root       4096 Apr  6  2022  DriveApp
drwx------ 2 root root       4096 Oct 26  2021  GDToT
-rw------- 1 root root    3017667 Dec 13  2014 'How to get started with Drive.pdf'


In [ ]:
!ls -lt '/content/drive/My Drive/TTS/'

total 11334218
-rw------- 1 root root 11606238907 Nov 14 17:51 whisper-medium-santali.zip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 10.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
